In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sn
from sklearn import metrics
import time
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('winequalityN.csv')
#df = df.fillna(0)
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
df_new = df.copy()

for i in df_new.columns:
    if df_new[i].isnull().sum() > 0:
        df_new[i] = df_new[i].fillna(df[i].mean())

In [4]:
df_bins = df_new.copy()
print(df_bins.shape)

df_bins.type = df_bins.type.map({'white':0, 'red':1})
bins = [0,4,7,10]

labels = [0,1,2] 

df_bins['quality_range'] = pd.cut(x = df_bins['quality'], bins = bins, labels = labels)
df_bins.head()

(6497, 13)


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_range
0,0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,1
1,0,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,1
2,0,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,1
3,0,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1
4,0,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1


In [5]:
#PCA before training
x = df_bins.iloc[:,:12]
y = df_bins['quality_range']
X = StandardScaler().fit_transform(x)
print("Before PCA:",X.shape)
pca = PCA(n_components=5)
X = pca.fit_transform(X)
print("After PCA:",X.shape)

Before PCA: (6497, 12)
After PCA: (6497, 5)


In [6]:
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state = 42,train_size = 0.7)

In [7]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(4547, 5) (4547,)
(1950, 5) (1950,)


In [8]:
def decTreeScore2(crit = 'gini',  maxDepth = 3 , minSamples = 1, minSplit = 5):
    dect = DecisionTreeClassifier(criterion = crit, max_depth = maxDepth, min_samples_leaf = minSamples, 
                                 min_samples_split = minSplit, random_state= 42)
    
    dect.fit(x_train, y_train)
    
    accuracy = accuracy_score(y_test, dect.predict(x_test))
    
    y_pred = dect.predict(x_test)
    a = confusion_matrix(y_test,y_pred)
    print(accuracy)
    print(a)
    
    return accuracy,a 

In [9]:
decTreeScore2()

0.9338461538461539
[[   2   67    0]
 [   5 1818    4]
 [   0   53    1]]


(0.9338461538461539,
 array([[   2,   67,    0],
        [   5, 1818,    4],
        [   0,   53,    1]], dtype=int64))

In [10]:
# show the decision tree model
# import graphviz and sklearn.tree first
from sklearn import tree
import graphviz
from graphviz import Source
from sklearn.tree import DecisionTreeClassifier


dect = DecisionTreeClassifier(criterion = 'gini', max_depth = 3, min_samples_leaf = 1, 
                                 min_samples_split = 5, random_state= 42)

dect.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [11]:
y_pred = dect.predict(x_test) # let the model predict the test data

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9338461538461539

In [12]:
start_time = time.time()
X = pca.fit_transform(X)
end_time = time.time()
print("-----%seconds-----" % (end_time - start_time))

-----0.0029921531677246094econds-----
